In [25]:
import os
import json
import time
import anthropic

from typing import Dict, Any, List
from anthropic.types.messages.batch_create_params import Request
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming

In [ ]:
from dotenv import load_dotenv

load_dotenv("/home/ubuntu/LLM-tutorials/keys.env")
anthropic_api_key = os.environ.get("ANTHROPIC_API_KEY")
print(anthropic_api_key)

client = anthropic.Anthropic(api_key=anthropic_api_key)

In [11]:
def get_structured_json_from_claude(input_text: str, 
                                   json_format: Dict[str, Any],
                                   model: str = "claude-3-7-sonnet-20250219") -> Dict[str, Any]:
    """
    Claude API를 호출하여 구조화된 JSON 응답을 받는 함수
    
    Args:
        input_text: 사용자 입력 텍스트
        json_format: 원하는 JSON 출력 형식 (예시 구조)
        model: 사용할 Claude 모델
        
    Returns:
        Dict: Claude가 생성한 JSON 응답
    """
    
    # JSON 형식 예시를 문자열로 변환
    json_format_str = json.dumps(json_format, ensure_ascii=False, indent=2)
    
    # 프롬프트 구성
    system_prompt = f"""당신은 정보를 구조화된 JSON으로 반환하는 AI 어시스턴트입니다.
사용자 입력을 분석하고 다음 JSON 형식에 정확히 맞춰 응답하세요:

{json_format_str}

다른 설명이나 텍스트 없이 오직 JSON만 반환하세요. JSON은 유효한 형식이어야 합니다."""

    # API 호출 
    message = client.messages.create(
        model=model,
        max_tokens=1024,
        system=system_prompt,
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    
    # 응답에서 JSON 추출
    response_text = message.content[0].text
    
    # JSON 파싱
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        # JSON 부분만 추출 시도 (모델이 추가 텍스트를 출력한 경우)
        import re
        json_pattern = r'({[\s\S]*})'
        match = re.search(json_pattern, response_text)
        if match:
            try:
                return json.loads(match.group(1))
            except:
                return {"error": "JSON 파싱 실패", "raw_response": response_text}
        else:
            return {"error": "응답에서 JSON을 찾을 수 없음", "raw_response": response_text}

In [12]:
customer_feedback = """3년 동안 충성 사용자였지만, 최근 UI 업데이트는 재앙입니다. 
기본 기능을 찾는 것이 보물찾기가 되었네요. 게다가 새로운 '프리미엄' 가격은 터무니없습니다. 
이것이 빨리 해결되지 않으면 다른 서비스로 전환할 것을 고려 중입니다."""

In [ ]:
# 원하는 JSON 출력 형식 정의
json_schema = {
    "sentiment": "string (긍정/부정/중립)",
    "key_issues": ["문제점 목록"],
    "action_items": [
        {
            "team": "담당 팀",
            "task": "수행해야 할 작업"
        }
    ]
}

# API 호출 및 결과 출력
result = get_structured_json_from_claude(customer_feedback, json_schema)
print(json.dumps(result, ensure_ascii=False, indent=2))

In [ ]:
## 두 개의 별도 요청이 비동기 처리를 위해 함께 배치됩니다. 각 요청에는 고유한 custom_id가 있으며 Messages API 호출에 사용할 표준 매개변수가 포함되어 있습니다.
## Message Batch의 processing_status 필드는 배치가 처리 중인 단계를 나타냅니다. in_progress로 시작하여 배치의 모든 요청 처리가 완료되고 결과가 준비되면 ended로 업데이트됩니다.

message_batch = client.messages.batches.create(
    requests=[
        Request(
            custom_id="my-first-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{
                    "role": "user",
                    "content": "Hello, world",
                }]
            )
        ),
        Request(
            custom_id="my-second-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{
                    "role": "user",
                    "content": "Hi again, friend",
                }]
            )
        )
    ]
)

print(message_batch)

In [ ]:
message_batch = client.messages.batches.retrieve("msgbatch_015xMn3PEfJ4CDNpFdjz7FVP")
print(f"Batch {message_batch.id} processing status is {message_batch.processing_status}")

In [ ]:
# 메모리 효율적인 청크로 결과 파일을 스트리밍하여 한 번에 하나씩 처리
for result in client.messages.batches.results("msgbatch_015xMn3PEfJ4CDNpFdjz7FVP"):
    match result.result.type:
        case "succeeded":
            print(f"Success! {result.custom_id}")
        case "errored":
            if result.result.error.type == "invalid_request":
                # 재전송하기 전에 요청 본문을 수정해야 함
                print(f"Validation error {result.custom_id}")
            else:
                # 요청을 직접 재시도할 수 있음
                print(f"Server error {result.custom_id}")
        case "expired":
            print(f"Request expired {result.custom_id}")

In [ ]:
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

client = anthropic.Anthropic()

message_batch = client.messages.batches.create(
    requests=[
        Request(
            custom_id="my-first-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                system=[
                    {
                        "type": "text",
                        "text": "You are an AI assistant tasked with analyzing literary works. Your goal is to provide insightful commentary on themes, characters, and writing style.\n"
                    },
                    {
                        "type": "text",
                        "text": "<the entire contents of Pride and Prejudice>",
                        "cache_control": {"type": "ephemeral"}
                    }
                ],
                messages=[{
                    "role": "user",
                    "content": "Analyze the major themes in Pride and Prejudice."
                }]
            )
        ),
        Request(
            custom_id="my-second-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                system=[
                    {
                        "type": "text",
                        "text": "You are an AI assistant tasked with analyzing literary works. Your goal is to provide insightful commentary on themes, characters, and writing style.\n"
                    },
                    {
                        "type": "text",
                        "text": "<the entire contents of Pride and Prejudice>",
                        "cache_control": {"type": "ephemeral"}
                    }
                ],
                messages=[{
                    "role": "user",
                    "content": "Write a summary of Pride and Prejudice."
                }]
            )
        )
    ]
)
